In [1]:
libraries = c("dplyr", "magrittr", "tidyr", "surveillance", "rstan") 
for(x in libraries) { library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE) }

'%&%' = function(x,y)paste0(x,y)

R.Version()$version.string

This is surveillance 1.18.0. For overview type ‘help(surveillance)’.

rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)



[1] "R version 3.4.4 (2018-03-15)"

# Loading the dataset

In [2]:
datestar = as.Date("2020-05-10")
datemin = as.Date("2019-12-25") # particular choice

read.csv("../data/JapaneseDataCOVID19 ("%&%format(datestar,"%y%m%d")%&%").csv") %>%
    mutate(report = if_else(is.na(confirmed), reported, confirmed), report = 
           as.Date(report), onset = as.Date(onset)) %>%
    select(-confirmed, -reported) -> df

(tstar = as.numeric(datestar - datemin))

df %<>% mutate(time_onset = as.numeric(onset - datemin),  
               time_report = as.numeric(report - datemin))

df %>% head

Warning message in `[<-.factor`(`*tmp*`, i, value = structure(c(1L, 1L, 1L, 1L, :
“invalid factor level, NA generated”


[1] 137

,exp_type,onset,is_asymptomatic,report,time_onset,time_report
,<fct>,<date>,<int>,<date>,<dbl>,<dbl>
1,imported,2020-01-03,0,NA,9,NA
2,imported,2020-01-14,0,NA,20,NA
3,imported,2020-01-21,0,NA,27,NA
4,imported,2020-01-23,0,NA,29,NA
5,imported,2020-01-22,0,NA,28,NA
6,imported,2020-01-26,0,NA,32,NA


In [3]:
# Changing it to wide format respectively to time of illness onset
df %>% select(exp_type,time_onset) %>% group_by(exp_type,time_onset) %>% summarize(n=n()) %>%
    spread(exp_type,n) %>% 
    right_join(expand_grid(time_onset = seq(from = 1, to = tstar-1, by = 1)), by='time_onset') %>% 
    replace(is.na(.), 0) %>%
    rename(time = time_onset) -> Df_cases
Df_cases %>% tail

time,domestic,imported
<dbl>,<int>,<int>
131,30,0
132,19,0
133,18,0
134,8,0
135,5,0
136,0,0


In [4]:
# the same for cases only with observed report date
df %>% filter(is.na(time_onset)) %>% select(exp_type,time_report) %>% group_by(exp_type,time_report) %>% 
    summarize(n=n()) %>% spread(exp_type,n) %>% rename(imported_unobs=imported, domestic_unobs=domestic, time=time_report) %>% 
    right_join(Df_cases, by='time') %>% replace(is.na(.), 0) -> Df_cases
Df_cases %>% tail

time,domestic_unobs,imported_unobs,domestic,imported
<dbl>,<int>,<int>,<int>,<int>
131,33,0,30,0
132,0,0,19,0
133,0,0,18,0
134,0,0,8,0
135,0,0,5,0
136,0,0,0,0


## <font color="purple">Backprojection</font>

In [5]:
# here due to stability of the backprojection algorithm we add zero counts beyond tstar
df_cases = Df_cases %>% rbind(data.frame(time=seq(tstar,tstar+10,1), imported=0, imported_unobs=0, domestic=0, domestic_unobs=0))
df_cases %>% tail

time,domestic_unobs,imported_unobs,domestic,imported
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
142,0,0,0,0
143,0,0,0,0
144,0,0,0,0
145,0,0,0,0
146,0,0,0,0
147,0,0,0,0


In [6]:
K = nrow(df_cases)

# Incubation time
inc_fit = list(meanlog=1.519, sdlog=0.615)
incubation_probability = plnorm(1:K, inc_fit$meanlog, inc_fit$sdlog) - plnorm(1:K-1, inc_fit$meanlog, inc_fit$sdlog)
inc_pmf = c(0,incubation_probability[1:41])
k_used = c(2,2) #parameter of smoothing primary vs secondary

epsilon = 1e-3

## import
if (sum(df_cases$imported)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$imported)
    bpnp.control = list(k = k_used[1], eps = rep(1e-4,2), iter.max=rep(1000,2), 
                        Tmark = tstar-1,  ##attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("R","C"))
    sts_bp = backprojNP(sts, incu.pmf=inc_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    # output result
    df_cases['imported_backproj'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['imported_backproj'] = df_cases['imported_backproj']/sum(df_cases['imported_backproj'])*sum(df_cases$imported)
} else 
    df_cases['imported_backproj'] = 0

## secondary
if (sum(df_cases$domestic)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$domestic)
    bpnp.control = list(k = k_used[2], eps = rep(1e-4,2), iter.max=rep(1000,2), 
                        Tmark = tstar-1, ##attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("R","C"))
    sts_bp = backprojNP(sts, incu.pmf=inc_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    df_cases['domestic_backproj'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['domestic_backproj'] = df_cases['domestic_backproj']/sum(df_cases['domestic_backproj'])*sum(df_cases$domestic)
} else
    df_cases['domestic_backproj'] = 0
df_cases %<>% ungroup

df_cases %>% tail(30)

time,domestic_unobs,imported_unobs,domestic,imported,imported_backproj,domestic_backproj
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
118,0,0,155,0,0,98.262752729
119,0,0,162,0,0,94.086335109
120,87,0,106,0,0,87.094886897
121,0,0,148,0,0,76.768161336
122,0,0,115,0,0,63.569456097
123,0,0,105,0,0,49.086724506
124,0,0,120,0,0,34.978929279
125,105,0,94,0,0,22.030147212
126,63,0,69,0,0,11.532751350


## <font color="purple">Backprojection including cases with unobserved date of illness onset</font>

In [7]:
# first backprojecting to the suggesting date of illness onset

# Reporting delay: from onset to confirmation
onset2report_fit = list(param1 = 1.741, param2 = 8.573)
onset2report_probability = pweibull(1:K, onset2report_fit$param1, scale=onset2report_fit$param2) - pweibull(1:K-1, onset2report_fit$param1, scale=onset2report_fit$param2)
onset2report_pmf = c(0,onset2report_probability[1:41])

## import
if (sum(df_cases$imported_unobs)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$imported_unobs)
    bpnp.control = list(k = k_used[1], eps = rep(1e-4,2), iter.max=rep(1000,2), 
                        Tmark = tstar - 1, #attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("R","C"))
    sts_bp = backprojNP(sts, incu.pmf=onset2report_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    # output result
    df_cases['imported_backproj_unobs'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['imported_backproj_unobs'] = df_cases['imported_backproj_unobs']/sum(df_cases['imported_backproj_unobs'])*sum(df_cases$imported_unobs)
} else 
    df_cases['imported_backproj_unobs'] = 0

## secondary
if (sum(df_cases$domestic_unobs)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$domestic_unobs)
    bpnp.control = list(k = k_used[2], eps = rep(1e-3,2), iter.max=rep(1000,2), 
                        Tmark = tstar - 1, #attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("C"))
    sts_bp = backprojNP(sts, incu.pmf=onset2report_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    df_cases['domestic_backproj_unobs'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['domestic_backproj_unobs'] = df_cases['domestic_backproj_unobs']/sum(df_cases['domestic_backproj_unobs'])*sum(df_cases$domestic_unobs)
} else
    df_cases['domestic_backproj_unobs'] = 0
df_cases %<>% ungroup

df_cases %>% filter(time<tstar) %>% tail(20)

time,domestic_unobs,imported_unobs,domestic,imported,imported_backproj,domestic_backproj,imported_backproj_unobs,domestic_backproj_unobs
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
117,0,0,219,0,0,1.023578e+02,0,29.642559150
118,0,0,155,0,0,9.826275e+01,0,33.563791188
119,0,0,162,0,0,9.408634e+01,0,30.860008874
120,87,0,106,0,0,8.709489e+01,0,23.965418928
121,0,0,148,0,0,7.676816e+01,0,16.273634715
122,0,0,115,0,0,6.356946e+01,0,9.325428153
123,0,0,105,0,0,4.908672e+01,0,4.142751315
124,0,0,120,0,0,3.497893e+01,0,1.312018116
125,105,0,94,0,0,2.203015e+01,0,0.270393156


In [8]:
# Now backprojecting unobs + obs by the incubation period

## import
if (sum(df_cases$imported_unobs)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$imported_backproj_unobs + df_cases$imported)
    bpnp.control = list(k = k_used[1], eps = rep(1e-4,2), iter.max=rep(1000,2), Tmark = nrow(df_cases)-10,
                        Tmark = tstar - 1, #attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("R","C"))
    sts_bp = backprojNP(sts, incu.pmf=inc_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    # output result
    df_cases['imported_backproj_incl_unobs'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['imported_backproj_incl_unobs'] = df_cases['imported_backproj_incl_unobs']/sum(df_cases['imported_backproj_incl_unobs'])*sum(df_cases$imported_backproj_unobs + df_cases$imported)
} else 
    df_cases['imported_backproj_incl_unobs'] = 0

## secondary
if (sum(df_cases$domestic_unobs)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$domestic_backproj_unobs + df_cases$domestic)
    bpnp.control = list(k = k_used[2], eps = rep(1e-4,2), iter.max=rep(1000,2), 
                        Tmark = tstar - 1, #attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("C"))
    sts_bp = backprojNP(sts, incu.pmf=inc_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    df_cases['domestic_backproj_incl_unobs'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['domestic_backproj_incl_unobs'] = df_cases['domestic_backproj_incl_unobs']/sum(df_cases['domestic_backproj_incl_unobs'])*sum(df_cases$domestic_backproj_unobs + df_cases$domestic)
} else
    df_cases['domestic_backproj_incl_unobs'] = 0
df_cases %<>% ungroup

df_cases %>% filter(time<tstar) %>% tail(30)

time,domestic_unobs,imported_unobs,domestic,imported,imported_backproj,domestic_backproj,imported_backproj_unobs,domestic_backproj_unobs,imported_backproj_incl_unobs,domestic_backproj_incl_unobs
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
107,147,2,371,2,0.003023249,2.543770e+02,0.020587073,42.292808881,0.001834398,237.92465785
108,171,0,356,0,0.000000000,2.316336e+02,0.003143117,26.357013390,0.000000000,223.34958953
109,0,0,283,2,0.000000000,2.062971e+02,0.000000000,17.213670206,0.000000000,207.56303035
110,69,2,324,0,0.000000000,1.832082e+02,0.000000000,11.543523423,0.000000000,194.85996321
111,0,0,293,0,0.000000000,1.660957e+02,0.000000000,7.734397165,0.000000000,188.50835171
112,83,0,280,0,0.000000000,1.541181e+02,0.000000000,5.542600623,0.000000000,186.37905562
113,115,0,226,0,0.000000000,1.443115e+02,0.000000000,5.335891853,0.000000000,183.09501718
114,124,0,225,0,0.000000000,1.339834e+02,0.000000000,7.740862381,0.000000000,173.48355883
115,0,0,216,0,0.000000000,1.218354e+02,0.000000000,13.222509400,0.000000000,155.43056961


In [9]:
sum(df_cases$domestic) + sum(df_cases$domestic_unobs)

[1] 13865

In [10]:
sum(df_cases$domestic_unobs)

[1] 1657

In [11]:
df_cases %<>% select(-domestic_backproj_unobs,-imported_backproj_unobs) %>% filter(time<tstar)
df_cases %>% write.csv("../results/JapaneseDataCOVID19_with_backproj ("%&%format(datestar,"%y%m%d")%&%").csv", row.names=FALSE)

# <font color="green">[R] Effective reproduction number</font>

## <font color="purple">Stan program</font>

In [12]:
## The code is written by Andrei R. Akhmetzhanov and verified by Kenji Mizumoto
"functions {
    /* discretesized version of lognormal distribution */
    vector plognormal(real mu, real sigma, int K) {
        vector[K] res; 
        for (k in 1:K)
            res[k] = exp(lognormal_lcdf(k | mu, sigma));

        return append_row(res[1], res[2:K]-res[1:(K-1)]);        
    }

    /* discretesized version of Weibull distribution */
    vector pweibull(real kappa, real theta, int K) {
        vector[K] res; 
        for (k in 1:K)
            res[k] = -expm1(-pow(1.0*k/theta, kappa));

        return append_row(res[1], res[2:K]-res[1:(K-1)]);  
    }

    /* calculating the convolutions */
    // X: first function, Yrev: reversed version of the second function
    // K: length of X and Yrev
    // the result is the vector of length K-1, because the first element equal zero is omitted
    vector convolution(vector X, vector Yrev, int K) {
        vector[K-1] res;
        res[1] = X[1]*Yrev[K];
        for (k in 2:K-1) 
            res[k] = dot_product(head(X, k), tail(Yrev, k));

        return res;        
    }

    // Special form of the convolution with adjusting to the delay
    // F: cumulative distribution function of the reporting delay
    vector convolution_with_delay(vector X, vector Yrev, vector F, int K) {
        vector[K-1] res;
        vector[K] Z = X ./ F;

        res[1] = F[2]*Z[1]*Yrev[K];
        for (k in 3:K) 
            res[k-1] = F[k]*dot_product(head(Z, k-1), tail(Yrev, k-1));

        return res;        
    }
}

data {
    int<lower = 1> K; //number of days
    vector<lower = 0>[K] imported_backproj;
    vector<lower = 0>[K] domestic_backproj;
    int<lower = K> upper_bound;

    // serial interval
    real<lower = 0> param1_SI;
    real<lower = 0> param2_SI;

    // reporting delay is given by Weibul distribution
    real<lower = 0> param1_delay;
    real<lower = 0> param2_delay;

    // incubation period
    real mu_inc;
    real<lower = 0> sigma_inc;
}

transformed data {
    vector[K] cases_backproj;

    vector[K-1] conv;
    vector[K-1] conv_delay_adj;

    cases_backproj = imported_backproj + domestic_backproj;
    {
        // serial interval
        vector[K] gt = pweibull(param1_SI, param2_SI, K);
        vector[K] gtrev;
 
        vector[upper_bound] IncPeriod = plognormal(mu_inc, sigma_inc, upper_bound); 
        vector[upper_bound] IncPeriod_inv;
        vector[upper_bound] ReportDelay;
        vector[upper_bound] conv_report_inc;
        
        vector[upper_bound] F_tmp;
        vector[K] F; 
        
        for (k in 1:K) 
            gtrev[k] = gt[K+1-k];
        
        for (k in 1:upper_bound)
            IncPeriod_inv[k] = IncPeriod[upper_bound+1-k];

        // vector[upper_bound] ReportDelay;
        ReportDelay = pweibull(param1_delay, param2_delay, upper_bound);

        // convolution of the reporting delay distribution and incubation period
        // vector[upper_bound] conv_report_inc;
        conv_report_inc = append_row(rep_vector(0,1), convolution(ReportDelay, IncPeriod_inv, upper_bound));
        conv_report_inc = cumulative_sum(conv_report_inc);
        
        ReportDelay = cumulative_sum(ReportDelay);
        for (k in 1:upper_bound) 
            F_tmp[k] = ReportDelay[upper_bound+1-k];
        F = head(F_tmp, K);

        // convolution without adjusting for the delay
        conv = convolution(cases_backproj, gtrev, K);

        // convolution with adjusting for the delay
        conv_delay_adj = convolution_with_delay(cases_backproj, gtrev, F, K);
    }
}

parameters {
    // effective reproduction number without adjustment to the delay in reporting
    vector<lower = 0>[K-1] Rt;

    // effective reproduction number with adjustment to the delay in reporting
    vector<lower = 0>[K-1] Rt_adj;
}

model {
    Rt ~ normal(2.4, 2.0);
    Rt_adj ~ normal(2.4, 2.0);
    
    target += gamma_lpdf(tail(domestic_backproj, K-1) | Rt .* conv + 1e-13, 1.0)
            + gamma_lpdf(tail(domestic_backproj, K-1) | Rt_adj .* conv_delay_adj + 1e-13, 1.0);
}" %>% cat(file="fit_infection.stan", sep="", fill=TRUE)

In [13]:
#model discription
stanmodel = NULL
stanmodel = stan_model(file='fit_infection.stan')

## <font color="purple">Data</font>

In [14]:
stan_data = list(
  K = nrow(df_cases),
  imported_backproj = df_cases$imported_backproj,
  domestic_backproj = df_cases$domestic_backproj,
  upper_bound = tstar,
  ## Serial interval [Nishiura et al 2020 - only certain cases]
  param1_SI = 2.305,
  param2_SI = 5.452,
  ## reported delay
  param1_delay = onset2report_fit$param1,
  param2_delay = onset2report_fit$param2,
  ## incubation period [Linton et al 2020 - with right truncation excl. Wuhan residents]
  mu_inc = inc_fit$meanlog,
  sigma_inc = inc_fit$sdlog
  )

## <font color="purple">MCMC settings</font>

In [15]:
Iter =   10000
Warmup =  2000
Thin =      5
Chains =    2
Seed =   1234

## <font color="purple">Model fit</font>

In [16]:
fit <- sampling(stanmodel, 
                data=stan_data,
                iter=Iter,
                warmup=Warmup,
                thin=Thin, 
                chains = Chains, 
                seed=1234)


SAMPLING FOR MODEL 'fit_infection' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 7.4e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.74 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 10000 [  0%]  (Warmup)
Chain 1: Iteration: 1000 / 10000 [ 10%]  (Warmup)
Chain 1: Iteration: 2000 / 10000 [ 20%]  (Warmup)
Chain 1: Iteration: 2001 / 10000 [ 20%]  (Sampling)
Chain 1: Iteration: 3000 / 10000 [ 30%]  (Sampling)
Chain 1: Iteration: 4000 / 10000 [ 40%]  (Sampling)
Chain 1: Iteration: 5000 / 10000 [ 50%]  (Sampling)
Chain 1: Iteration: 6000 / 10000 [ 60%]  (Sampling)
Chain 1: Iteration: 7000 / 10000 [ 70%]  (Sampling)
Chain 1: Iteration: 8000 / 10000 [ 80%]  (Sampling)
Chain 1: Iteration: 9000 / 10000 [ 90%]  (Sampling)
Chain 1: Iteration: 10000 / 10000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 3.00449 seconds (Warm-up)
Chain 1:                6.34912 seconds (Sampling)
Ch

In [17]:
# Check the result
## Rt is for backproj
## Rt_adj is for backproj_incl_unobs
## The first and the last values should be ignored because no information is available and Rt* is unstable
summary(fit)$summary %T>% write.csv("../results/Results of Stan (script C).csv")

,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
Rt[1],3.8445595,0.028121755,1.6050452,1.01144610,2.6262266,3.7767560,4.9318587,7.110510,3257.540,0.9994510
Rt[2],3.8177748,0.029216368,1.6178876,1.04443632,2.6119955,3.7506648,4.9216659,7.171970,3066.509,1.0000382
Rt[3],3.8084007,0.030137285,1.6423889,0.94584722,2.6214755,3.7231035,4.8656102,7.247571,2969.913,1.0001665
Rt[4],3.8748129,0.028548326,1.6191433,1.01226224,2.6764219,3.7427467,4.9727646,7.270018,3216.691,0.9996841
Rt[5],3.8499597,0.030502838,1.6255927,0.99411555,2.6861667,3.7452740,4.9277641,7.238154,2840.161,1.0005493
Rt[6],3.8215746,0.029653974,1.6492048,0.94630242,2.6216301,3.7096665,4.8832403,7.355680,3093.024,0.9997500
Rt[7],3.8486133,0.028084792,1.5794598,1.09828572,2.6904791,3.7578372,4.8696974,7.249910,3162.822,0.9999978
Rt[8],3.7870620,0.028499755,1.5591764,1.04025628,2.6688748,3.6724432,4.8367421,7.151698,2993.011,0.9999709
Rt[9],3.7171770,0.027210851,1.5344630,1.00537089,2.6193221,3.6261803,4.6938281,6.969282,3180.011,0.9995780
Rt[10],3.7252126,0.028363869,1.6361554,0.85805210,2.5380915,3.6576992,4.7916843,7.135960,3327.501,0.9998643


In [18]:
# export result Rt with onset date to dataframe for visualization 
data.frame(summary(fit, pars="Rt")$summary) %>% select(mean, X2.5., X97.5.) %>% 
set_colnames(c("Rt", "lower", "upper")) %>%
mutate(onset=seq(datemin+1, by="days", length.out=nrow(df_cases)-1)) %T>% print() %>%
write.csv('../results/rtstan.csv')

# export result Rt_adj with onset date to dataframe for visualization 
data.frame(summary(fit, pars="Rt_adj")$summary) %>% select(mean, X2.5., X97.5.) %>%
set_colnames(c("Rt_adj", "lower", "upper")) %>%
mutate(onset=seq(datemin+1, by="days", length.out=nrow(df_cases)-1)) %T>% print() %>%
write.csv(., '../results/rtadjstan.csv')

            Rt       lower      upper      onset
1   3.84455950 1.011446103 7.11050965 2019-12-26
2   3.81777481 1.044436318 7.17196999 2019-12-27
3   3.80840074 0.945847223 7.24757087 2019-12-28
4   3.87481287 1.012262242 7.27001834 2019-12-29
5   3.84995972 0.994115551 7.23815380 2019-12-30
6   3.82157458 0.946302425 7.35567981 2019-12-31
7   3.84861333 1.098285716 7.24990973 2020-01-01
8   3.78706199 1.040256284 7.15169801 2020-01-02
9   3.71717697 1.005370892 6.96928230 2020-01-03
10  3.72521256 0.858052101 7.13595991 2020-01-04
11  2.46037630 0.450592666 5.30995188 2020-01-05
12  3.04283484 0.631672889 6.07524296 2020-01-06
13  3.52494673 0.905733270 6.82251155 2020-01-07
14  3.67600368 0.937193882 7.05931062 2020-01-08
15  3.72021222 0.942888673 6.99549232 2020-01-09
16  3.58455978 0.865679642 6.84464537 2020-01-10
17  3.11240469 0.728072011 6.12367983 2020-01-11
18  2.42175895 0.429487746 5.19301857 2020-01-12
19  2.51338639 0.472718582 5.20436516 2020-01-13
20  2.97862757 0.736